In [14]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import csv
import ast


In [2]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.2)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.9
Method: manual login


In [3]:
#RUN ONLY IF YOUR NAMES ARE IN A CSV, IF NOT MOVE TO NEXT CELL
def read_names_from_csv(file_path, encoding='utf-8'):
    all_names = []

    with open(file_path, 'r', encoding=encoding, errors='ignore') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Assuming the first column contains first names and the second column contains last names
        for row in csv_reader:
            try:
                if len(row) >= 2:  # Ensure the row has at least two columns
                    first_names = row[1]
                    last_names = row[3]
                    full_name = first_names + " " + last_names
                    all_names.append(full_name)
            except  UnicodeDecodeError as e:
                print(f"Error decoding line {csv_reader.line_num}: {e}")

    return all_names

# Example usage
file_path = r'\Users\ashle\OneDrive\School\Capstone\names\original_names.csv'
all_names = read_names_from_csv(file_path)
all_names = all_names[1:]

# Now you can use the lists first_names and last_names as needed
print(f'ALL NAMES({len(all_names)}) :')
print("======================================================")
print(all_names)


ALL NAMES(4216) :
['Thomas James', 'Simon Karpatkin', 'Philip Majerus', 'Ralph Nachman', 'Yale Nemerson', 'Andrew Wallace', 'Chester Alper', 'Vincent Andriole', 'Stanley Appel', 'Elia Ayoub', 'Fritz Bach', 'Kurt Bloch', 'Paul Bornstein', 'Lewis Braverman', 'Bayard Clarkson', 'Herbert Cooper', 'John Dietschy', 'Chester Edelmann', 'John Ensinck', 'Yale Enson', 'Harvey Feigenbaum', 'Daniel Foster', 'Emil Freireich', 'Thomas Gaffney', 'Mehran Goulian', 'Arthur Grollman', 'Paul Gulyassy', 'Donald Harrison', 'Peter Holt', 'Richard Hong', 'Richard Hyde', 'Norman Javitt', 'Richard Johnson', 'Manuel Kaplan', 'Nicholas Kefalides', 'Hermes Kontos', 'Claude Lenfant', 'Gilbert Levinson', 'Ruven Levitan', 'Harvey Marver', 'Hugh McDevitt', 'James McGuigan', 'William Mellman', 'Charles Mengel', 'Thomas Merigan', 'Ralph Morris', 'Beverley Murphy', 'Harold Neu', 'C. Osterland', 'Robert Palmer', 'Frank Parker', 'Claude Reed', 'Morris Reichlin', 'Jack Remington', 'Jesse Roth', 'Stuart Schlossman', 'Leslie

In [ ]:
#RUN CELL IF YOUR NAMES ARE IN A LIST
#put your list of names here
all_names = []

In [4]:
#gets research id for a name
def researcherIds(name):
    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    listDict = res.json['researchers']
    allIds = set()
    for idv in listDict:
        allIds.add(idv['id'])

    allIds = list(allIds)  # Convert set back to a list if needed
    
    if len(allIds) > 512:    
        allIds = allIds[0:512]
    print(f'Total of {len(allIds)} unique ids for {name} ')

    # Constructing the DSL query with the list of IDs
    query_ids = ', '.join([f'"{id_val}"' for id_val in allIds])
    query = f'search researchers where id in [{query_ids}] return researchers[id+obsolete+redirect]'

    # Execute the DSL query
    res2 = dsl.query_iterative(query)
    listDict2 = res2.json['researchers']
    allWorkingIDs = set()
    for person in listDict2:
        if person['obsolete'] == 0:  # Current Working Id(s)
            allWorkingIDs.add(person["id"])
        else:
            for ids in person['redirect']:
                allWorkingIDs.add(ids)
    
    print(f'++++++++ FINAL {len(allWorkingIDs)} Working ID(s) +++++++++++')  
    print(allWorkingIDs)
    
    return allWorkingIDs
testIDs = researcherIds("Leslie Schoenfield")

Starting iteration with limit=1000 skip=0 ...
0-18 / 18 (0.70s)
===
Records extracted: 18
Starting iteration with limit=1000 skip=0 ...


Total of 18 unique ids for Leslie Schoenfield 


0-18 / 18 (2.30s)
===
Records extracted: 18


++++++++ FINAL 2 Working ID(s) +++++++++++
{'ur.0102031123.62', 'ur.014424615212.32'}


In [5]:
def allIds(namesList):
    finalNames = {} 
    secondNames = {}
    zeroNames = {}
    count = 0

    for name in namesList:
        try:
            validIds = researcherIds(name)

            if len(validIds) == 0:
                zeroNames[name] = set()
                finalNames[name] = set()
                pass
            elif len(validIds) == 1:
                count += 1
                print(name, count)
                finalNames[name] = validIds
            elif len(validIds) == 2:
                secondNames[name] = validIds
                finalNames[name] = validIds
            else:
                finalNames[name] = validIds

        except Exception as e:
            print(f"An error occurred for {name}: {e}")
    return finalNames, secondNames, zeroNames
        
resultAll, pairsAll, zeroNames = allIds(all_names)
#resultAll is all names with more than two ids
#pairsAll is all names with two ids
#zeroNames is all names with no ids

In [ ]:
#this writes only names with more than two ids to file
print(len(resultAll))
print(len(pairsAll))
print(len(zeroNames))

def write_set_to_csv(file_path, my_set):
    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        # Write the header
        writer.writerow(['Name', 'ID'])

        # Write the data
        for name, id_set in my_set.items():                
            id_set = list(id_set)
            writer.writerow([name, id_set]) 
            
write_set_to_csv("dimensionsAI_NameIds.csv", resultAll)


In [ ]:
#if you want to write all you names to a csv file
merged_dict = {resultAll, pairsAll, zeroNames}
write_set_to_csv("dimensionsAI_NameIds.csv", merged_dict)